In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 18394190
paper_name = 'ruotolo_ottonello_2008' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [8]:
original_data = pd.read_excel('raw_data/gb-2008-9-4-r67-S2.xlsx', sheet_name='Additional file 2', skiprows=2)

In [9]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 388 x 15


In [10]:
original_data.head()

,ORF,Cd (*),Scoring rate (#),Ni (*),Scoring rate (#).1,ORF name,Description,ORF type,Overlap with dubious ORFs,Cd microarray data [7],Ionome Ni data [77],BlastP (Best Hit),H. sapiensorthologs,Human disease,GO ID
0,YAL002W,-4,3/5,2,5/5,VPS8,Membrane-associated protein that interacts wit...,Verified,------------,1.04173,-----------------,"AFR002C (Ashbya gossypii, 1E-173)",Vacuolar protein sorting-associated protein 8 ...,--------------------,GO:0045324
1,YAL007C,2,3/5,2,3/5,ERP2,Protein that forms a heterotrimeric complex wi...,Verified,------------,0.859331,-----------------,"YOR016C (Saccharomyces cerevisiae, 6E-58)",Transmembrane emp24 domain containing protein ...,--------------------,GO:0006888
2,YAL010C,-2,3/5,0,---,MDM10,Subunit of both the Mdm10-Mdm12-Mmm1 complex a...,Verified,------------,0.977212,-----------------,"AEL335C (Ashbya gossypii, 2E-98)",-------------,--------------------,GO:0000001 GO:0000002 GO:0000723 GO:0006461 GO...
3,YAL021C,-4,4/5,-2,3/5,CCR4,Component of the CCR4-NOT transcriptional comp...,Verified,------------,-------------------,-----------------,"ACR288W (Ashbya gossypii, 1E-261)",Carbon catabolite repressor protein 4 homolog ...,--------------------,GO:0000288 GO:0000289 GO:0000289 GO:0006357 GO...
4,YAL026C,-2,3/5,-2,3/5,DRS2,Aminophospholipid translocase (flippase) that ...,Verified,------------,1.16089,-----------------,"ENSP00000371078 (Homo sapiens, 1E-261)",Probable phospholipid transporting ATPase (EC_...,--------------------,GO:0000028 GO:0006886 GO:0006892 GO:0006892 GO...


In [11]:
original_data['orf'] = original_data['ORF'].astype(str)

In [12]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [13]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [14]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [ORF, Cd (*), Scoring rate (#), Ni (*), Scoring rate (#).1, ORF name, Description, ORF type, Overlap with dubious ORFs, Cd microarray data [7], Ionome Ni data [77], BlastP (Best Hit), H. sapiensorthologs, Human disease, GO ID, orf]
Index: []


In [15]:
original_data.set_index('orf', inplace=True)

In [16]:
original_data = original_data[['Cd (*)','Ni (*)']].copy()

In [17]:
original_data = original_data.apply(pd.to_numeric, axis=1, errors='coerce')

In [18]:
original_data = original_data.groupby(original_data.index).mean()

In [19]:
original_data.shape

(388, 2)

# Load & process tested strains

In [20]:
tested = pd.read_excel('raw_data/List of tested yeast mutant strains.xlsx', sheet_name='Foglio1')

In [21]:
tested.head()

,ORF name,strain,batch,plate,row,column
0,YAL002W,BY4739,01_1,101,E,1
1,YAL004W,BY4739,01_1,101,D,11
2,YAL005C,BY4739,01_1,101,D,12
3,YAL007C,BY4739,01_1,101,D,10
4,YAL008W,BY4739,01_1,101,D,9


In [22]:
tested['orf'] = tested['ORF name'].astype(str)

In [23]:
tested['orf'] = clean_orf(tested['orf'])

In [24]:
tested['orf'] = translate_sc(tested['orf'], to='orf')

In [25]:
# Make sure everything translated ok
t = looks_like_orf(tested['orf'])
print(tested.loc[~t,])

Empty DataFrame
Columns: [ORF name, strain, batch, plate, row, column, orf]
Index: []


In [26]:
tested_orfs = tested['orf'].unique()

In [27]:
missing = [orf for orf in original_data.index.values if orf not in tested_orfs]
missing

['YCL005W-A', 'YCL058W-A', 'YLR027C', 'YOL148C']

In [28]:
tested_orfs = list(tested_orfs) + missing

In [29]:
original_data = original_data.reindex(index=tested_orfs, fill_value=0)

# Prepare the final dataset

In [30]:
data = original_data.copy()

In [31]:
dataset_ids = [1317, 1318]
datasets = datasets.reindex(index=dataset_ids)

In [32]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [33]:
data.head()

dataset_id,1317,1318
data_type,value,value
orf,,
YAL002W,-4,2
YAL004W,0,0
YAL005C,0,0
YAL007C,2,2
YAL008W,0,0


## Subset to the genes currently in SGD

In [34]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 26


In [35]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,1317,1318
,data_type,value,value
gene_id,orf,,
2,YAL002W,-4,2
1863,YAL004W,0,0
4,YAL005C,0,0
5,YAL007C,2,2
6,YAL008W,0,0


# Normalize

In [36]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [37]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [38]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,1317,1318,1317,1318
,data_type,value,value,valuez,valuez
gene_id,orf,,,,
2,YAL002W,-4,2,-4.532815,3.691526
1863,YAL004W,0,0,0.023009,0.036915
4,YAL005C,0,0,0.023009,0.036915
5,YAL007C,2,2,2.300921,3.691526
6,YAL008W,0,0,0.023009,0.036915


# Print out

In [39]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [40]:
from IO.save_data_to_db3 import *

In [41]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/2 [00:00<?, ?it/s]

Deleting all datasets for PMID 18394190...
Inserting the new data...


100%|██████████| 2/2 [00:14<00:00,  7.45s/it]

Updating the data_modified_on field...
